# Automatic derivation of CCSD theory

This notebook serves as an example of interactive usage of drudge for complex symbolic manipulations in Jupyter notebooks.  Here we can see how the classical CCSD theory can be derived automatically.

## Preparatory work

First, we need to set up the Spark environment.  Here we just use parallelization on the local machine.

In [ ]:
from pyspark import SparkContext
ctx = SparkContext('local[*]', 'ccsd')

Or we can also use the dummy spark to emulate the Spark environment in a purely serial way.  Note that we need just **one** Spark context.  These two cells should not be both evaluated.

In [ ]:
from dummy_spark import SparkContext
ctx = SparkContext()

With the Spark context, we can construct the drudge specific for this problem.  Then we can define some names that is going to be used frequently.

In [ ]:
from sympy import *
from drudge import *

dr = PartHoleDrudge(ctx)
p = dr.names

c_ = p.c_
c_dag = p.c_dag
a, b = p.V_dumms[:2]
i, j = p.O_dumms[:2]

## Cluster excitation operator

Here, we by using the Einstein summation convention tensor creator, we can just define the cluster operator in a way very similar to how we would writen them down on paper.

In [ ]:
t1 = IndexedBase('t^1')
t2 = IndexedBase('t^2')

clusters = dr.einst(
    t1[a, i] * c_dag[a] * c_[i] +
    Rational(1, 4) * t2[a, b, i, j] * c_dag[a] * c_dag[b] * c_[j] * c_[i]
)

We can have a peek at the cluster operator.

In [ ]:
clusters.display()

Now we need tell the system about the symmetry on $t^2$, so that it can be used in simplification.

In [ ]:
dr.set_dbbar_base(t2, 2)

## Similarity transform of the Hamiltonian

Here we can use a loop to nest the commutation conveniently.  And IPython magic can be used to time the operation.  Note that after the simplification, we explicitly redistribute the terms in the transformed Hamiltonian for better parallel performance in later operations.  Note that `drudge` does not automatically cache the result of tensor computations.  The `cache` method should be called explicitly when a tensor is going to be used multiple times.

In [ ]:
%%time

curr = dr.ham
h_bar = dr.ham
for order in range(0, 4):
    curr = (curr | clusters).simplify() * Rational(1, order + 1)
    curr.cache()
    h_bar += curr
h_bar.repartition(cache=True)

The transformed Hamiltonian can be very complex.  Instead of reading its terms, we can just have a peek by get a count of the number of terms it contains.

In [ ]:
h_bar.n_terms

## Working equation derivation

With the similarity transformed Hamiltonian, we are now ready to derive the actual working equations.  First, the energy equation can be derived by taking the vacuum expectation value of the transformed Hamiltonian.

In [ ]:
en_eqn = h_bar.eval_fermi_vev().simplify()

We can have a look at its contents to see if it is what we would expect.

In [ ]:
en_eqn.display()

Next, we can create a projector to derive the working equation for the singles amplitude.


In [ ]:
proj = c_dag[i] * c_[a]
t1_eqn = (proj * h_bar).eval_fermi_vev().simplify()

In the same way, we can display its content.

In [ ]:
t1_eqn.display()

The working equation for the doubles amplitude can be done in the same way, just it can be slower.

In [ ]:
%%time

proj = c_dag[i] * c_dag[j] * c_[b] * c_[a]
t2_eqn = (proj * h_bar).eval_fermi_vev().simplify()

Since the equation can be slightly complex, we can vaguely sort the terms in increasing complexity before display them.

In [ ]:
t2_eqn = t2_eqn.sort()
t2_eqn.display()